# Customizing shooting moves

The tutorial focuses on how to customize shooting moves in the move scheme. We discuss three major reasons to do that:

1. To use a different kind of shooting move, such as two-way shooting, when needed.
2. To use a different shooting point selector, such as a Gaussian biased selector, to get better efficiency.
3. To create move schemes that only sample part of the network, for example, to perform TIS with each ensemble sampled in parallel.

In addition, you'll learn about:

* Creating a "setup" file with engine and other simulation information.
* Using the OpenPathSampling command line interface (CLI) to run simulations from the setup file.

Note that you'll need to install the CLI, which can be done with either `pip install openpathsampling-cli` or `conda install -c conda-forge openpathsampling-cli`.


In [ ]:
import matplotlib.pyplot as plt

import openpathsampling as paths
from openpathsampling import strategies

## A simple toy model example system

In this notebook, we'll use a simple toy system. For the most part, the ideas here directly generalize to other engines, although we'll make a few comments where the units associated with OpenMM require special care.

We'll start by loading a number things from the storage file:

In [ ]:
storage = paths.Storage("2_state_toy.nc", mode='r')
state_A = storage.volumes['A']
state_B = storage.volumes['B']
cv = storage.cvs['x']
engine = storage.engines['toy_engine']
initial_conditions = storage.tags['initial_conditions']

The toy model here uses a simple 2D potential energy surface, described by:

$$V(x, y) = x^6 + y^6 - e^{-12(x+0.6)^2 - 5 y^2} - e^{-12(x-0.6)^2 - 5 y^2}$$

We can visualize the toy engine using the `toy_plot_helpers` included in this repository:

In [ ]:
%run toy_plot_helpers.py

pes = engine.topology.pes

plot = ToyPlot()
plot.contour_range = np.arange(-1.5, 1.0, 0.1)
plot.add_pes(pes)
fig = plot.plot() 

We run this simulation at a temperature of $T=0.1$, so the barrier between those wells is about $10\ k_\text{B} T$.

## Changing the shooting move type (e.g., two-way shooting)

TPS is usually introduced with two-way shooting. In that algorithm, you select a frame of the trajectory, make some modification (typically changing the velocities in some way consistent with the thermodynamic ensemble), and then integrate the equations of motion forward and backward.

However, this is problematic if the velocity memory is short compared to your TPS trajectory length, as is often the case in condensed phase systems. In this case, each shot might as well be a committor shot. The ideal place to shoot from is the where the committor is 1/2, where each shot has only a 50% chance of landing in the correct state. But this gives you a maximum of a 25% acceptance rate.

To get around this, we typically use the one-way shooting algorithm when working with condensed matter systems. One-way shooting works under the assumption that you are using a stochastic integrator. If that's the case, then instead of changing the velocities at the shooting point, you can use the fact that the integrator will generate a new sequence of random numbers. Therefore, you create a new trajectory by only running in one direction, and that trajectory is still a valid, physical trajectory.

In general, OpenPathSampling defaults to using one-way shooting. However, there are circumstances where you might want to use the older two-way shooting algorithm. For example, if the velocity memory is shorter than the period between saved frames, it might be better to use two-way shooting. If you intend to use deterministic dynamics, then one-way shooting is not possible.

This part of the tutorial will show you how to use two-way shooting instead of one-way shooting in OPS. There are two ways of doing this: either replace the existing one-way shooting strategy, or create a new move scheme from scratch.

In [ ]:
tps_network = paths.TPSNetwork(initial_states=state_A, final_states=state_B)

### Replacing parts of an existing move scheme

Frequently, the easiest way to modify a move scheme (especially a complex move scheme) is to replace the parts that you want to change. In this case, we can start with the move scheme that only includes 1-way shooting, and work from there.

In [ ]:
modified_scheme = paths.OneWayShootingMoveScheme(
    network=tps_network,
    engine=engine
).named("2_way_from_1_way")

For our two-way shooting, we'll need to create a `TwoWayShootingStrategy`. This will require a modifier; to keep things simple, we'll completely randomize velocities (consistent with a given temperature) using `paths.RandomVelocities`. In order to be consistent between engines, `RandomVelocities` takes its input as the inverse temperature, $\beta = 1/(k_\text{B}T)$.

For the toy engine, we can obtain the temperature from `engine.integ.temperature`, and we work in units where $k_\text{B}=1$, so it is easy to calculate $\beta$. For other engines, you'll need to use the correct value of $k_\text{B}$. For OpenMM, you also need to worry about units: use `simtk.unit.BOLTZMANN_CONSTANT_kB`.



In [ ]:
print(engine.integ.temperature)

In [ ]:
# YOUR TURN: Set beta correctly (yes, this is as easy as you think).
# beta = ... # fill in the ellipsis and uncomment this line

In [ ]:
modifier = paths.RandomVelocities(beta=beta, engine=engine)
shooting_strategy = strategies.TwoWayShootingStrategy(modifier=modifier, engine=engine)

To modify a move scheme, just `append` new strategies. In this case, the new strategy for the group of moves called `'shooting'` will be overwritten. If you gave the `TwoWayShootingStrategy` a different string for its `group` argument, then you would create a second group of movers, and each move would have a 50/50 chance of using one-way shooting or of using two-way shooting.

In [ ]:
modified_scheme.append(shooting_strategy)

### Building a move scheme from scratch

In this particular case, our overall move scheme will not be too complicated. In fact, let's start by looking at the at the code for 

In [ ]:
import inspect
from IPython.display import Code

Code(inspect.getsource(paths.OneWayShootingMoveScheme))

The output above is the full source code for the `OneWayShootingMoveScheme`. You can see that all it does is pass the network up to the superclass's initialization, and then `append` two move strategies to itself. So we can recreate this:

In [ ]:
two_way_scheme = paths.MoveScheme(network=tps_network).named("2_way")
global_strategy = strategies.OrganizeByMoveGroupStrategy()
two_way_scheme.append([shooting_strategy, global_strategy])

## Changing the shooting point selection

A common problem in one-way shooting is that paths do not decorrelate quickly enough. This typically happens when there is a significant barrier withing the transition region, such that shooting points before the barrier always go back to the initial state (so only backward shots are accepted) and shooting points after the barrier always to do the final state (so only forward shots are accepted).

The same issue manifests in two-way shooting as a significant decrease in the acceptance rate for the shooting move. One side of the barrier always creates $A\to A$ trials, while the other side always creates $B\to B$ trials. With fewer $A\to B$ trials, the acceptance rate drops.

In [ ]:
import numpy as np
xvals = np.arange(-1.0, 1.0, 0.01)

def plot_gaussian_bias(selector):
    alpha = selector.alpha
    x_0 = selector.l_0
    gaussians = np.exp(-alpha*(xvals - x_0)**2)
    plt.plot(xvals, gaussians)

In [ ]:
biased_shooting_scheme = paths.MoveScheme(network=tps_network).named("biased_shooting")

In [ ]:
gaussian_sel = paths.GaussianBiasSelector(cv, alpha=100.0, l_0=0.0)

In [ ]:
plot_gaussian_bias(gaussian_sel)

In [ ]:
paths.GaussianBiasSelector?

In [ ]:
# YOUR TURN: Create a move scheme using a two-way shooting strategy with this selector.
# 1. Create a two-way shooting strategy that uses this Gaussian selector. (Use the 
#    selector keyword argument of TwoWayShootingStrategy.)
# 2. Append that strategy and the global_strategy to the biased_shooting_scheme


Next we verify that the initial conditions will work for the move schemes we've created.

In [ ]:
_ = two_way_scheme.initial_conditions_from_trajectories(initial_conditions)

In [ ]:
_ = biased_shooting_scheme.initial_conditions_from_trajectories(initial_conditions)

## Run the simulations

Now that we've created these various move schemes, we'll save them to a file, and use that as the input file for the OpenPathSampling command line interface.

In [ ]:
shooting_setup = paths.Storage("shooting_setup.nc", mode='w')

In [ ]:
# saving everything will take a few minutes
shooting_setup.tags['initial_conditions'] = initial_conditions
shooting_setup.save(two_way_scheme)
shooting_setup.save(biased_shooting_scheme)
shooting_setup.close()

Now, use the OpenPathSampling command line interface to run simulations with these. Run at least 500 steps (`-n 500`), or run more if you'd like.

```
$ openpathsampling pathsampling shooting_setup.nc -o 2_way.nc --scheme 2_way -n 500

$ openpathsampling pathsampling shooting_setup.nc -o biased.nc --scheme biased_shooting -n 500
```

Those simulations will take a few minutes, so this is a good time to take a quick break. In the next section, we'll analyze the results.

## Comparing two-way shooting to biased shooting

Let's open the 

In [ ]:
std_two_way = paths.Storage("2_way.nc", mode='r')
biased_two_way = paths.Storage("biased.nc", mode='r')

In [ ]:
two_way_scheme = std_two_way.schemes['2_way']
biased_scheme = biased_two_way.schemes['biased_shooting']

In [ ]:
# each of these will take about a minute
two_way_scheme.move_summary(std_two_way.steps)

In [ ]:
biased_scheme.move_summary(biased_two_way.steps)

As you can see, the acceptance rate for uniform 2-way shooting is pretty low. We improve this significantly by using the Gaussian biased shooting (keeping in mind that 25% is the theoretical maximum acceptance rate for the approach we use here.)

Next we will plot the shooting points with uniform shooting point selection and with the Gaussian biased shooting point selection. First we write and use a little function to extract the shooting points.

In [ ]:
def get_shooting_points(steps):
    """Function to extract x,y positions of all shooting points"""
    shooting_snaps = [step.change.canonical.details.shooting_snapshot for step in steps]
    xy = [snap.xyz[0][:2] for snap in shooting_snaps]  # get x and y positions
    return tuple(zip(*xy))  # [[x1, y1], [x2, y2]]  => ([x1, x2], [y1, y2])

In [ ]:
# the 0th step saves initial conditions, so we only have shooting moves as of step 1
std_x, std_y = get_shooting_points(std_two_way.steps[1:])
biased_x, biased_y = get_shooting_points(biased_two_way.steps[1:])

The next two plots show the location of shooting points in the when using uniform shooting (first plot) and when using Gaussian biased shooting (second plot). Note that the shooting points fall in a much narrower range in the Gaussian biased plots. This is also why we get a better acceptance rate!

In [ ]:
plot.plot()
plt.scatter(std_x, std_y, c=range(len(std_x)), cmap='rainbow');

In [ ]:
plot.plot()
plt.scatter(biased_x, biased_y, c=range(len(std_x)), cmap='rainbow');

## (Embarassingly) parallel TIS without replica exchange

The OPS `DefaultScheme` is designed to provide reasonable default behaviors for TIS. These include replica exchange moves, path reversal moves, the minus interface move, as well as shooting. In general, replica exchange TIS is a much more efficient way to sample than TIS without replica exchange. However, replica exchange TIS is much harder to parallelize, because the duration of each trajectory is not known before running the trajectory.

Therefore, in some cases you may want to sample each interface independently. This allows a naïve parallelization, since each interface is its own simulation.

Here, we will make a RETIS scheme that is similar to the default scheme, except it doesn't include the minus interface move.

In [ ]:
interfaces = paths.VolumeInterfaceSet(cv=cv, minvals=float("-inf"), 
                                      maxvals=[-0.60, -0.5, -0.4, -0.3, -0.2])
tis_network = paths.MISTISNetwork([(state_A, interfaces, state_B)]).named("tis")

In [ ]:
# this is basically the DefaultScheme without the minus interface move
retis_scheme = paths.MoveScheme(network=tis_network).named("retis")
retis_scheme.append([
    strategies.OneWayShootingStrategy(engine=engine),
    strategies.PathReversalStrategy(),
    strategies.NearestNeighborRepExStrategy(),
    strategies.OrganizeByMoveGroupStrategy()
])

The `OneWayShootingStrategy` includes an `ensembles`, which selects specific ensembles. The (normal TIS) ensembles sampled by the TIS network are in the attribute `sampling_ensembles`. (Aside: other ensembles, such as the minus interface ensembles, are in the `special_ensembles` attribute.) This means that we can create a strategy 

In [ ]:
ens_0_strategy = strategies.OneWayShootingStrategy(
    ensembles=[tis_network.sampling_ensembles[0]]
)

From here, we can make a 

In [ ]:
scheme_0 = paths.MoveScheme(tis_network).named("scheme_0")
scheme_1 = paths.MoveScheme(tis_network).named("scheme_1")
scheme_2 = paths.MoveScheme(tis_network).named("scheme_2")
scheme_3 = paths.MoveScheme(tis_network).named("scheme_3")
scheme_4 = paths.MoveScheme(tis_network).named("scheme_4")

In [ ]:
global_strategy = strategies.OrganizeByMoveGroupStrategy()

In [ ]:
# YOUR TURN: Make the correct strategies and append things to the scheme
# 1. Create a OneWayShootingStrategy for each ensemble
# 2. Append the global_strategy and the appropriate shooting strategy to each scheme


## Running TIS

Again, we'll use the command line interface to run the TIS. So the first stage is to save the relevant things to a file, and then we can use the `--scheme` option in the `pathsampling` command to select which scheme to run.

First, we check that all our schemes are correct:

In [ ]:
_ = retis_scheme.initial_conditions_from_trajectories(initial_conditions)

In [ ]:
_ = scheme_0.initial_conditions_from_trajectories(initial_conditions)

In [ ]:
_ = scheme_1.initial_conditions_from_trajectories(initial_conditions)

In [ ]:
_ = scheme_2.initial_conditions_from_trajectories(initial_conditions)

In [ ]:
_ = scheme_3.initial_conditions_from_trajectories(initial_conditions)

In [ ]:
_ = scheme_4.initial_conditions_from_trajectories(initial_conditions)

In [ ]:
# saving everything will take a few minutes
parallel_setup = paths.Storage("parallel_setup.nc", mode='w')
parallel_setup.tags['initial_conditions'] = initial_conditions
parallel_setup.save(retis_scheme)
parallel_setup.save(scheme_0)
parallel_setup.save(scheme_1)
parallel_setup.save(scheme_2)
parallel_setup.save(scheme_3)
parallel_setup.save(scheme_4)
parallel_setup.close()

In OPS, each individual move, such as an attempt to swap a specific pair of replicas, counts as a Monte Carlo step. So in order to make a fair comparison of the approaches with an without replica exchange, we want to ensure that they both have about the same number of shooting moves for each ensemble.

However, the `MoveScheme` can give us an estimate of how many total moves are required to get a certain number of moves of a certain mover. To get 250 trials of the 0th (only!) shooting mover in `scheme_0`, how many total steps do we need?

In [ ]:
scheme_0.n_steps_for_trials(scheme_0.movers['shooting'][0], 250)

That answer is probably pretty obvious. But what about our RETIS scheme? How many total steps to we need (on average) to get 250 trials of the 0th shooting mover from that move scheme?

In [ ]:
# YOUR TURN: Answer the question above


This should be a significantly larger number, and it is due to the many replica exchange and path reversal moves in that move scheme.

Now let's run the simulations. First, we equilibrate the initial condition. You can do this with:

```
$ openpathsampling equilibrate parallel_setup.nc -o retis_equil.nc --scheme retis --extra-steps 50
```

That will first run until the first decorrelated path (no frames in common with the initial trajectory), and then run an additional 50 MC steps. The results will be saved in the `scheme_0_equil.nc` file.

Then you can run the full simulation with:
```
$ openpathsampling pathsampling retis_equil.nc -o retis.nc -n $NSTEPS > retis.out &
```
where you should replace `$NSTEPS` with the number of steps you found for RETIS above. 

If you're not familiar, the `> retis.out` redirects the output to the file `retis.out` (you can `cat retis.out` to see progress updates), and the `&` at the end of the command forces the command to run in the background, so that you can issue more commands from the same command line (i.e., run multiple things in parallel).

* Why don't you need to specify a `--scheme` with the second command? (Hint: use the `openpathsampling contents` command on `retis_equil.nc` and `parallel_setup.nc`. How many move schemes are saved in each?) 


Do the same for `scheme_0`, `scheme_1`, `scheme_2`, `scheme_3` and `scheme_4`, running the path sampling with 250 steps each. In this way, you will be running all 5 interfaces in parallel.

## Analyzing the TIS simulations

In [ ]:
from openpathsampling.analysis import tis

In [ ]:
storage_0 = paths.Storage('scheme_0.nc', mode='r')
storage_1 = paths.Storage('scheme_1.nc', mode='r')
storage_2 = paths.Storage('scheme_2.nc', mode='r')
storage_3 = paths.Storage('scheme_3.nc', mode='r')
storage_4 = paths.Storage('scheme_4.nc', mode='r')
storage_retis = paths.Storage('retis.nc', mode='r')

In [ ]:
scheme_0 = storage_0.schemes['scheme_0']
scheme_1 = storage_1.schemes['scheme_1']
scheme_2 = storage_2.schemes['scheme_2']
scheme_3 = storage_3.schemes['scheme_3']
scheme_4 = storage_4.schemes['scheme_4']
scheme_retis = storage_retis.schemes['retis']

### Comparing the move summaries

In [ ]:
# takes a few minutes
scheme_retis.move_summary(storage_retis.steps)

In [ ]:
scheme_retis.move_summary(movers='shooting')

In [ ]:
scheme_retis.move_summary(movers='repex')

In [ ]:
# each of these takes a minute or so
scheme_0.move_summary(storage_0.steps)

In [ ]:
scheme_1.move_summary(storage_1.steps)

In [ ]:
scheme_2.move_summary(storage_2.steps)

In [ ]:
scheme_3.move_summary(storage_3.steps)

In [ ]:
scheme_4.move_summary(storage_4.steps)

In [ ]:
scheme_0.move_summary(movers='shooting')

### TIS analysis (crossing probabilities, etc.)

We don't actually have the flux here, so we can't calculate the actual rates. However, we can create a fake flux that says that the flux through the out of state $A$ and through the innermost interface is `1.0`. This allows us to use the rest of the `StandardTISAnalysis` object. It just means that the rate that gets reported is actually the total transition probability.

You can get the actual flux either from including a minus interface move in your TIS simulation, or from using direct MD. The `paths.TrajectoryTransitionAnalysis` class will analyze existing MD trajectories, or the `paths.DirectSimulation` class can run MD and analyze the flux on the fly.

In [ ]:
network = storage_retis.networks[0]
state_A = storage_retis.volumes['A']
interface_0 = network.sampling_transitions[0].interfaces[0]
fake_flux = tis.DictFlux({(state_A, interface_0): 1.0})

Finally, we assemble the `StandardTISAnalysis` and perform the analysis:

In [ ]:
%%time
# takes about 2 minutes
retis_analysis = tis.StandardTISAnalysis(
    network=network,
    flux_method=fake_flux,
    max_lambda_calcs={t: {'bin_width': 0.025, 'bin_range': (-0.6, 0.6)}
                      for t in network.sampling_transitions},
    steps=storage_retis.steps
)

Currently, the parallel analysis needs an extra step to run correctly. We need to create the `weighted_trajectories` object from the steps, and then perform the overall analysis using that as the input, instead of the steps themselves.

In [ ]:
%%time
# currently we need to manually join the weighted trajectories from each storage
# Future versions of OPS will simplify this
weighted_trajectories = {}
storages = [storage_0, storage_1, storage_2, storage_3, storage_4]
for storage, ensemble in zip(storages, network.sampling_ensembles):
    weighted_trajectories.update(
        tis.core.steps_to_weighted_trajectories(storage.steps, [ensemble])
    )

In [ ]:
%%time
# this will take a few minutes
state_A = storage_0.volumes['A']
interface_0 = network.sampling_transitions[0].interfaces[0]
fake_flux = tis.DictFlux({(state_A, interface_0): 1.0})
parallel_analysis = tis.StandardTISAnalysis(
    network=network,
    flux_method=fake_flux,
    max_lambda_calcs={t: {'bin_width': 0.025, 'bin_range': (-0.6, 0.6)}
                      for t in network.sampling_transitions},
    combiners={t.interfaces: paths.numerics.WHAM(cutoff=0.01,
                                                 interfaces=t.interfaces.lambdas)
               for t in network.sampling_transitions}
)
parallel_analysis.results['flux'] = fake_flux.calculate('foo')
parallel_analysis.results = parallel_analysis.from_weighted_trajectories(weighted_trajectories)

### Plotting the crossing probabilities

One of the spot-checks to see if your simulation is converged is to plot the crossing probabilities functions. For each ensemble, the `StandardTISAnalysis` calculates a crossing probability along the order parameter, defined as the fraction of paths in that ensemble that reach at least the given value on the $x$ axis. As such, the crossing probability is always 1 for values less than the cutoff for the interface. Additionally, two ensemble crossing probabilities should never cross; the one from an outer interface should always be higher at a given value of the order parameter than one from an inner interface.

There is also the *total* crossing probability, which is generated by using a histogram combining algorithm (usually WHAM) to combine the individual ensemble crossing probabilities into a good estimate for the true crossing probability (from the innermost interface). Like all crossing probabilities, this should be monotonically decreasing; if it is not, that is a sign of insufficient sampling.

Since the y-axis is probability, and we're looking at rare events, we frequently plot crossing probabilities on a semi-log plot.

In [ ]:
for ensemble in network.transitions[(state_A, state_B)].ensembles:
    crossing = retis_analysis.crossing_probability(ensemble)
    label = "Interface at $x$={:3.2f}".format(ensemble.lambda_i)
    plt.plot(crossing.x, crossing, label=label)

tcp_AB = retis_analysis.total_crossing_probability[(state_A, state_B)]
plt.plot(tcp_AB.x, tcp_AB, lw=2, color='k', label="Total crossing probability")
plt.legend()
plt.yscale('log')
plt.xlabel('$x$')
plt.ylabel('Crossing probability');

In [ ]:
for ensemble in network.transitions[(state_A, state_B)].ensembles:
    crossing = parallel_analysis.crossing_probability(ensemble)
    label = "x={:3.2f}".format(ensemble.lambda_i)
    plt.plot(crossing.x, crossing, label=label)

tcp_AB = parallel_analysis.total_crossing_probability[(state_A, state_B)]
plt.plot(tcp_AB.x, tcp_AB, lw=2, color='k', label="Total crossing probability")
plt.legend()
plt.yscale('log')
plt.xlabel('$x$')
plt.ylabel('Crossing probability');